In [1]:
import jieba
import os
import pandas as pd
import numpy as np
import csv
import re
import chardet
import emoji
import sys
import unicodedata
import joblib

In [3]:
#!pip3 install pandarallel
from pandarallel import pandarallel

#并行初始化
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## copy的载入情感词典和停用词典

In [4]:
affect_col_list = ['PA','PE','PD','PH','PG','PB','PK',
                   'NA','NB','NJ','NH','PF','NI','NC',
                   'NG','NE','ND','NN','NK','NL','PC',
                  'MH','MS','MA','MD','ME',
                  'P','N','Ne']
def load_affect_dict(filepath):
    m_affectdict = []
    for m_col in affect_col_list:
        m_col = []
        m_affectdict.append(m_col)
    for m_line in open(filepath,'r',encoding='utf-8').readlines():
        m_line = m_line.strip()
        kwd = m_line.split('\t')[0].strip()
        col = m_line.split('\t')[1].strip()
        m_affectdict[affect_col_list.index(col)].append(kwd)
    return m_affectdict

#载入情感词典
affect_dict_file = 'D:\my research\dictionary\dict_affect.txt'
affect_dict = load_affect_dict(affect_dict_file)

In [5]:
#创建停用词list
def load_stopwords(filepath):
    m_stopwords = [line.strip()for line in open(filepath,'r',encoding='utf-8').readlines()]
    return m_stopwords

#载入停用词表
stop_word_file = 'D:\my research\dictionary\stop_words_cn.txt'
stopwords = load_stopwords(stop_word_file)

In [6]:
# stopwords为停用词list
#stopwords = [line.strip() for line in open('stop.txt', 'r', encoding='utf-8').readlines()]

## 读入数据

In [7]:
#读取数据
group_1 = pd.read_csv('D:\my research\group\group_1.csv',encoding='utf-8')
group_1.columns = ['0','content','date','time','user_id']
group_1 = group_1.drop(columns='0')

In [8]:
#def cut_word(word):
#    cw = jieba.cut(word,cut_all=True)
#    return list(cw)

#group_1['cut_word'] = group_1['content'].apply(cut_word)

#pd.Series(group_1['cut_word'].sum()).value_counts()



In [9]:
#group_1.head()

In [10]:
#group_1.to_csv('D:\my research\group\group_1_cut.csv',encoding="utf-8_sig")

In [11]:
practise = group_1[0:100]
practise.head()

content        date  \
0  #上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...  2022-04-24   
1  #上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...  2022-04-23   
2  #上海疫情##上海民生#      无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...  2022-04-22   
3        #上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。   2022-04-22   
4  明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...  2022-04-21   

                  time     user_id  
0  2022-04-24 08:33:43  1000074972  
1  2022-04-23 05:46:14  1000074972  
2  2022-04-22 11:47:31  1000074972  
3  2022-04-22 09:41:33  1000074972  
4  2022-04-21 08:54:29  1000074972

In [12]:
practise.dtypes

content    object
date       object
time       object
user_id     int64
dtype: object

## 分词

In [13]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [14]:
from pandarallel import pandarallel
practise['cut_content'] = practise.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 0.637 seconds.
Prefix dict has been built successfully.
C:\Users\Administrator\AppData\Local\Temp/ipykernel_11392/1728287109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practise['cut_content'] = practise.content.apply(chinese_word_cut)


In [15]:
practise.head()

content        date  \
0  #上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...  2022-04-24   
1  #上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...  2022-04-23   
2  #上海疫情##上海民生#      无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...  2022-04-22   
3        #上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。   2022-04-22   
4  明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...  2022-04-21   

                  time     user_id  \
0  2022-04-24 08:33:43  1000074972   
1  2022-04-23 05:46:14  1000074972   
2  2022-04-22 11:47:31  1000074972   
3  2022-04-22 09:41:33  1000074972   
4  2022-04-21 08:54:29  1000074972   

                                         cut_content  
0  # 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 #   上海 返乡 潮 开...  
1  # 上海 疫情 ## 上海 民生 ## 上海 现状 #   . 上海 的 现状 为什么 会 ...  
2  # 上海 疫情 ## 上海 民生 #             无间道 ， 罗生门 ， 可能 ...  
3  # 上海 民生 ## 上海 疫情 ## 上海 团长 #   还是 你 牛 逼 呀 ， 团长 ...  
4  明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...

In [16]:
#def find_chinese(file):
#    pattern = re.compile(r'[^\u4e00-\u9fa5]')
#    chinese = re.sub(pattern, '', file)
#    print(chinese)
 
#def find_unchinese(file):
#    pattern = re.compile(r'[\u4e00-\u9fa5]')
#    unchinese = re.sub(pattern,"",file)
#    print(unchinese)

## 修改停用词表

In [17]:
type(stopwords)

list

In [18]:
stopwords[0]='!'
stopwords.append('##')
stopwords.append('链接')
new = (['网页','链接','上海','罗一舟','肖战','朱一龙','任嘉伦','邓伦','微博','视频'])
stopwords.extend(new)

In [19]:
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',
 '一转眼'

## 尝试转换unicode 但是失败的

In [20]:
#str 和unicode的互转
#coding=utf-8
def to_unicode(unicode_or_str):
    if isinstance(unicode_or_str,str):
        value=unicode_or_str.decode('utf-8')
    else:
        value=unicode_or_str
    return value
def to_str(unicode_or_str):
    value=unicode_or_str.encode('utf-8')
    return value

In [21]:
practise['cut_content'] = to_unicode(practise.cut_content)
stopwords = to_unicode(stopwords)

C:\Users\Administrator\AppData\Local\Temp/ipykernel_11392/219021477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practise['cut_content'] = to_unicode(practise.cut_content)


## 对数据框内文本进行分词加去除停用词，通过定义函数

In [22]:
# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stop = stopwords # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stop:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

out = []
for line in practise['content']:
    line_seg = seg_sentence(line) # 这里的返回值是字符串
    out.append(line_seg)

In [23]:
u_emoji = []
for line in out:
    line_u = str(line)
    line_un = emoji.replace_emoji(line_u,'') # 这里的返回值是字符串
    line_u = re.sub('[a-zA-Z]','',line_un) #去掉英文字母
    line_f = re.sub(r'[0-9]+', '', line_u) #去掉数字
    #line_ff = re.sub('([^\u4e00-\u9fa5])', '', line_f) #去掉除中文外全部字符
    u_emoji.append(line_f)

In [24]:
u_emoji

['疫情 民生 一刻 返乡   返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮 开启 滞留 人员 陆续 返乡 安徽 江苏 河南 返乡 人员 ',
 '疫情 民生 现状   现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 一条 汉奸 真的 太 抗战 那会 太 敏感 我乎 审核 说 一条 有人 发国难财 能量 不小 ',
 '疫情 民生             无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼一闭 当作 事情 发生 岁月 静好 叫唤 ',
 '民生 疫情 团长   牛 逼 团长 开个 车照 送 封控 小区 ',
 '明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇   市场监管   江 丄 孤 舟   中国新闻社 ',
 '疫情 民生 物资   黑市     ',
 '民生 疫情   外卖 骑手 好自为之 骑手 代购 袋 泡面  鸡蛋 收取  元 警方 调查 惊人 发现 最新消息 证实 赵某 因涉嫌 非法经营 罪 长宁 警方 依法 强制措施 案件 进一步 侦办 中 ',
 '民生 疫情   连花清 瘟 牛 逼 连花清 瘟 站台  收钱 办事 钱能 挣 一大批 粉丝 数十万 数百万  金  纷纷 连花清 瘟 站台 证实  收钱 连花清 瘟 站台 事实   ',
 '疫情 民生   救护车 谎称 进 小区 救人 买  条烟 业主 震怒 报 次警 近日 救护车 谎称 进 小区 救人 实则 买烟 新闻 引发 关注 ',
 '疫情 民生   跑腿 代购 牛 逼   ',
 '疫情 民生 刚刚 明白   网传 真的 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚 ',
 '疫情   核酸 检测 强烈建议 专业 事要 专业 人来 做 楼 里  未 拉走 居委会 老先生 老太太 志愿者 包括 毛估估 漏洞百出   ',
 '疫情 民生 有钱人 玩 事关 老百姓 民生 呼吁 政府部门 管管   ',
 '疫情 民生   不算 太贵 老板 还算 良心 关键 至少 送货 只会 送 烂 菜 枯叶 要强 

In [25]:
name = ['del']
test=pd.DataFrame(columns=name,data=u_emoji)

In [26]:
# 列拼接,默认是并集
tt = pd.concat([practise,test],axis=1)
tt.head()

content        date  \
0  #上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...  2022-04-24   
1  #上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...  2022-04-23   
2  #上海疫情##上海民生#      无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...  2022-04-22   
3        #上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。   2022-04-22   
4  明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...  2022-04-21   

                  time     user_id  \
0  2022-04-24 08:33:43  1000074972   
1  2022-04-23 05:46:14  1000074972   
2  2022-04-22 11:47:31  1000074972   
3  2022-04-22 09:41:33  1000074972   
4  2022-04-21 08:54:29  1000074972   

                                         cut_content  \
0  # 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 #   上海 返乡 潮 开...   
1  # 上海 疫情 ## 上海 民生 ## 上海 现状 #   . 上海 的 现状 为什么 会 ...   
2  # 上海 疫情 ## 上海 民生 #             无间道 ， 罗生门 ， 可能 ...   
3  # 上海 民生 ## 上海 疫情 ## 上海 团长 #   还是 你 牛 逼 呀 ， 团长 ...   
4  明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...   

                                                 del  
0  疫情 民生 一刻 返乡   返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮...  
1  疫情 民生 现状   现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 ...  
2  疫情 民生             无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼...  
3                   民生 疫情 团长   牛 逼 团长 开个 车照 送 封控 小区   
4  明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇   市场监管   江 丄 孤 ...

In [27]:
#pip install emoji

In [28]:
#def filter_emoji(desstr, restr=''):
    # 过滤表情
#    try:
        #co = re.sub(emoji.get_emoji_regexp(), r"", text)
#        co = re.compile(u'[\U00010000-\U0010ffff]')
#    except re.error:
#        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
#    return co.sub(restr, desstr)

In [29]:
#from emoji import get_emoji_regexp
#un_emoji = []
#for line in tt['del']:
#    line_un = filter_emoji(line) # 这里的返回值是字符串
#    un_emoji.append(line_un)

In [30]:
#def give_emoji_free_text(text):
#    allchars = [str for str in text.decode('utf-8')]
#    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
#    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
#    return clean_text

In [31]:
#uu_emoji = []
#for line in tt['del']:
#    line_un = emoji.demojize(line)# 这里的返回值是字符串
#    print(line_un)
#    line_u = emoji.emojize(line_un)
#    print(line_u)
    #uu_emoji.append(line_un)

In [32]:
#p = emoji.replace_emoji(u'A 🏌️‍♀️ is eating a 🥐','')
#print(p)

## copy 清华 提取关键词及其词频

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(tt['del']).toarray(), columns=vect.get_feature_names())
term_matrix.head()

一个月  一位  一刻  一周  一圈  一夜之间  一大堆  一大批  一套  一委  ...  鞋子  顺丰  顺遂  领导  验证  验车  \
0    0   0   1   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
1    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   1   0   0   
2    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
3    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
4    1   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   

   骑手  高估  鸡蛋  黑市  
0   0   0   0   0  
1   0   0   0   0  
2   0   0   0   0  
3   0   0   0   0  
4   0   0   0   0  

[5 rows x 696 columns]

In [34]:
vect = CountVectorizer(stop_words=frozenset(stopwords))

In [35]:
term_matrix = pd.DataFrame(vect.fit_transform(tt['del']).toarray(), columns=vect.get_feature_names())

D:\Program_Files\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [36]:
term_matrix.head()

一个月  一位  一刻  一周  一圈  一夜之间  一大堆  一大批  一套  一委  ...  鞋子  顺丰  顺遂  领导  验证  验车  \
0    0   0   1   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
1    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   1   0   0   
2    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
3    0   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   
4    1   0   0   0   0     0    0    0   0   0  ...   0   0   0   0   0   0   

   骑手  高估  鸡蛋  黑市  
0   0   0   0   0  
1   0   0   0   0  
2   0   0   0   0  
3   0   0   0   0  
4   0   0   0   0  

[5 rows x 696 columns]

In [37]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

term_matrix = pd.DataFrame(vect.fit_transform(tt['del']).toarray(), columns=vect.get_feature_names())
term_matrix.head()

D:\Program_Files\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


一刻  三顿  上班  仓鼠  代购  健健康康  健康  办公  可惜  同事  ...  结束  老板  能力  花草  解封  这是  邻居  \
0   1   0   0   0   0     0   0   0   0   0  ...   0   0   0   0   0   0   0   
1   0   0   0   0   0     0   0   0   0   0  ...   0   0   0   0   0   0   0   
2   0   0   0   0   0     0   0   0   0   0  ...   0   0   0   0   0   0   0   
3   0   0   0   0   0     0   0   0   0   0  ...   0   0   0   0   0   0   0   
4   0   0   0   0   0     0   0   0   0   0  ...   0   1   0   0   0   0   0   

   防疫  隔离  领导  
0   0   1   0  
1   0   0   1  
2   0   0   0  
3   0   0   0  
4   0   0   0  

[5 rows x 44 columns]

## 词频统计 尝试

In [38]:
u_emoji

['疫情 民生 一刻 返乡   返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮 开启 滞留 人员 陆续 返乡 安徽 江苏 河南 返乡 人员 ',
 '疫情 民生 现状   现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 一条 汉奸 真的 太 抗战 那会 太 敏感 我乎 审核 说 一条 有人 发国难财 能量 不小 ',
 '疫情 民生             无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼一闭 当作 事情 发生 岁月 静好 叫唤 ',
 '民生 疫情 团长   牛 逼 团长 开个 车照 送 封控 小区 ',
 '明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇   市场监管   江 丄 孤 舟   中国新闻社 ',
 '疫情 民生 物资   黑市     ',
 '民生 疫情   外卖 骑手 好自为之 骑手 代购 袋 泡面  鸡蛋 收取  元 警方 调查 惊人 发现 最新消息 证实 赵某 因涉嫌 非法经营 罪 长宁 警方 依法 强制措施 案件 进一步 侦办 中 ',
 '民生 疫情   连花清 瘟 牛 逼 连花清 瘟 站台  收钱 办事 钱能 挣 一大批 粉丝 数十万 数百万  金  纷纷 连花清 瘟 站台 证实  收钱 连花清 瘟 站台 事实   ',
 '疫情 民生   救护车 谎称 进 小区 救人 买  条烟 业主 震怒 报 次警 近日 救护车 谎称 进 小区 救人 实则 买烟 新闻 引发 关注 ',
 '疫情 民生   跑腿 代购 牛 逼   ',
 '疫情 民生 刚刚 明白   网传 真的 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚 ',
 '疫情   核酸 检测 强烈建议 专业 事要 专业 人来 做 楼 里  未 拉走 居委会 老先生 老太太 志愿者 包括 毛估估 漏洞百出   ',
 '疫情 民生 有钱人 玩 事关 老百姓 民生 呼吁 政府部门 管管   ',
 '疫情 民生   不算 太贵 老板 还算 良心 关键 至少 送货 只会 送 烂 菜 枯叶 要强 

In [39]:
new = ' '.join(u_emoji)
new

'疫情 民生 一刻 返乡   返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮 开启 滞留 人员 陆续 返乡 安徽 江苏 河南 返乡 人员  疫情 民生 现状   现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 一条 汉奸 真的 太 抗战 那会 太 敏感 我乎 审核 说 一条 有人 发国难财 能量 不小  疫情 民生             无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼一闭 当作 事情 发生 岁月 静好 叫唤  民生 疫情 团长   牛 逼 团长 开个 车照 送 封控 小区  明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇   市场监管   江 丄 孤 舟   中国新闻社  疫情 民生 物资   黑市      民生 疫情   外卖 骑手 好自为之 骑手 代购 袋 泡面  鸡蛋 收取  元 警方 调查 惊人 发现 最新消息 证实 赵某 因涉嫌 非法经营 罪 长宁 警方 依法 强制措施 案件 进一步 侦办 中  民生 疫情   连花清 瘟 牛 逼 连花清 瘟 站台  收钱 办事 钱能 挣 一大批 粉丝 数十万 数百万  金  纷纷 连花清 瘟 站台 证实  收钱 连花清 瘟 站台 事实    疫情 民生   救护车 谎称 进 小区 救人 买  条烟 业主 震怒 报 次警 近日 救护车 谎称 进 小区 救人 实则 买烟 新闻 引发 关注  疫情 民生   跑腿 代购 牛 逼    疫情 民生 刚刚 明白   网传 真的 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚 网传 嘉定 静默 天 取消 快递 嘉定区 防控 办 月   日   市 统一 部署   全区 综合 防控 措施   发起 清零 攻坚  疫情   核酸 检测 强烈建议 专业 事要 专业 人来 做 楼 里  未 拉走 居委会 老先生 老太太 志愿者 包括 毛估估 漏洞百出    疫情 民生 有钱人 玩 事关 老百姓 民生 呼吁 政府部门 管管    疫情 民生   不算 太贵 老板 还算 良心 关键 至少 送货 只会 送 烂 菜 枯叶 要强    疫情 民生 抗疫 物资   这是 真的 话 抓 枪毙      疫情 民生   民间 自发 团购 能

In [40]:
def feature_extraction(words):
    count = []
    #切词后词的总数
    cnt_tags = len(words)
    #print(cnt_tags)
     
    #提取每个情感词类的词频比率
    idx = 0
    for g_col in affect_col_list:
        
        #切词后，其中包含情感词的总数
        affect_cnt = 0
        
        #统计每个词类下关键词出现的总频次affect_cnt
        for i in range(cnt_tags):
            s = words[i]
            if(s in affect_dict[idx]):
                affect_cnt += 1
        
        #计算比率
        r_affect = 0.0
        if (cnt_tags > 0):
            r_affect = affect_cnt/cnt_tags
        
        new = str(affect_col_list[idx])+' '+str(affect_cnt)+' '+str(r_affect)
        count.append(new) 
        idx += 1
    
    return count

In [41]:
aaa = []
for line in u_emoji:
    line_u = str(line)
    line_uu = line_u.split()
    aaa.append(line_uu)

new=sum(aaa,[])

In [42]:
affect_cun = feature_extraction(new) # 这里的返回值是字符串

In [43]:
affect_cun 

['PA 32 0.023827252419955324',
 'PE 3 0.0022338049143708115',
 'PD 3 0.0022338049143708115',
 'PH 19 0.014147431124348473',
 'PG 2 0.0014892032762472078',
 'PB 4 0.0029784065524944155',
 'PK 6 0.004467609828741623',
 'NA 1 0.0007446016381236039',
 'NB 6 0.004467609828741623',
 'NJ 5 0.0037230081906180195',
 'NH 0 0.0',
 'PF 2 0.0014892032762472078',
 'NI 0 0.0',
 'NC 1 0.0007446016381236039',
 'NG 1 0.0007446016381236039',
 'NE 3 0.0022338049143708115',
 'ND 2 0.0014892032762472078',
 'NN 28 0.02084884586746091',
 'NK 0 0.0',
 'NL 0 0.0',
 'PC 2 0.0014892032762472078',
 'MH 20 0.014892032762472078',
 'MS 4 0.0029784065524944155',
 'MA 0 0.0',
 'MD 2 0.0014892032762472078',
 'ME 4 0.0029784065524944155',
 'P 0 0.0',
 'N 0 0.0',
 'Ne 0 0.0']

## 主题建模尝试

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
#!pip install spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
#读入数据
group_2_del = pd.read_csv('D:\my research\group\group_2\group_2_final.csv',encoding='utf-8')
group_2_del.columns = ['0','content','date','time','user_id','cut_content','del']
group_2_del = group_2_del.drop(columns='0')
group_2_del.head()

content        date  \
0  #上海民生##上海疫情##核酸#检测《上海人核酸“续命”尴尬：缺管子、乱加码、排队最长3小时...  2022-06-02   
1  #上海民生##上海疫情##上海复工#上海醒来，6月1号不会是哄孩子的吧。6月1日零时起 上海...  2022-05-30   
2  #上海民生##上海疫情# 有关团长《独家｜起底“团长江湖”：平价被抵制、供应商“强制”加价、...  2022-05-16   
3  #居委会##上海民生##上海疫情# 上海奇葩居委干部，假借抗疫强索私吞翡翠，官方属实》一个居...  2022-05-07   
4                #上海民生##上海疫情# 祝贺🎉五一劳动节，迎接双满月，你劳动了吗？   2022-05-01   

                  time     user_id  \
0  2022-06-02 13:22:28  1000074972   
1  2022-05-30 19:42:39  1000074972   
2  2022-05-16 09:38:37  1000074972   
3  2022-05-07 23:39:52  1000074972   
4  2022-05-01 17:34:27  1000074972   

                                         cut_content  \
0  # 上海 民生 ## 上海 疫情 ## 核酸 # 检测 《 上海 人 核酸 “ 续命 ” 尴...   
1  # 上海 民生 ## 上海 疫情 ## 上海 复工 # 上海 醒来 ， 6 月 1 号 不会...   
2  # 上海 民生 ## 上海 疫情 #   有关 团长 《 独家 ｜ 起底 “ 团长 江湖 ”...   
3  # 居委会 ## 上海 民生 ## 上海 疫情 #   上海 奇葩 居委 干部 ， 假借 抗...   
4  # 上海 民生 ## 上海 疫情 #   祝贺 🎉 五一劳动节 ， 迎接 双 满月 ， 你 ...   

                                                 del  
0  民生 疫情 核酸 检测 核酸 续命 尴尬 缺 管子 乱 加码 排队 最长 小时 核酸 续命 ...  
1  民生 疫情 复工 醒来 月 号 哄 孩子 月 日 零时   有序 恢复 住宅小区 出入 公共...  
2  民生 疫情   团长 独家 起底 团长 江湖 平价 抵制 供应商 强制 加价 黑白 团长 大...  
3  居委会 民生 疫情   奇葩 居委 干部 假借 抗疫 强索 私吞 翡翠 官方 属实 居委 干...  
4                      民生 疫情   祝贺  五一劳动节 迎接 双 满月 劳动

In [7]:
# 构造 TF-IDF
#tf_idf_vectorizer = TfidfVectorizer()
#tf_idf = tf_idf_vectorizer.fit_transform(tt['del'])
tf_idf_vectorizer = TfidfVectorizer(min_df = 0.001,max_df = 0.8)
tf_idf = tf_idf_vectorizer.fit_transform(group_2_del['del'].values.astype('U'))

In [8]:
# 特征词列表
feature_names = tf_idf_vectorizer.get_feature_names()
# 特征词 TF-IDF 矩阵
matrix = tf_idf.toarray()
feature_names_df = pd.DataFrame(matrix,columns=feature_names)
print(feature_names_df)
feature_names_df

        nan  一下子  一个多月  一个月   一人   一件   一份   一会  一会儿   一位  ...   高考   魅力   魔幻  \
0       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...     ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
174203  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174204  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174205  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174206  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174207  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

         鱼币   鸡蛋   麻烦   黄瓜 

nan  一下子  一个多月  一个月   一人   一件   一份   一会  一会儿   一位  ...   高考   魅力   魔幻  \
0       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...     ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
174203  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174204  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174205  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174206  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
174207  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

         鱼币   鸡蛋   麻烦   黄瓜   黑暗   鼓励   龚俊  
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...     ...  ...  ...  ...  ...  ...  ...  
174203  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
174204  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
174205  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
174206  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
174207  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[174208 rows x 1480 columns]

In [9]:
# 指定 lda 主题数
n_topics = 5
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20

lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=5, random_state=0)

In [10]:
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [50]:
# 计算 n_top_words 个主题词
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)
top_words_df
# 保存 n_top_words 个主题词到 csv 文件中
#top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf.toarray()

# 计算完毕主题概率分布情况
predict_df = predict_to_data_frame(lda, X)
predict_df
# 保存文本主题概率分布到 csv 文件中
#predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)

P(topic 1)  P(topic 2)  P(topic 3)  P(topic 4)  P(topic 5)
0     0.051159    0.051350    0.795131    0.051134    0.051226
1     0.036899    0.036703    0.852769    0.036805    0.036825
2     0.039718    0.039852    0.039754    0.840954    0.039722
3     0.059192    0.059356    0.059370    0.762916    0.059166
4     0.049717    0.800956    0.049696    0.049777    0.049855
..         ...         ...         ...         ...         ...
95    0.668273    0.082917    0.082924    0.082951    0.082935
96    0.052380    0.052354    0.052355    0.052366    0.790546
97    0.073314    0.073301    0.073293    0.706780    0.073313
98    0.817572    0.045743    0.045540    0.045543    0.045603
99    0.813676    0.046672    0.046550    0.046572    0.046530

[100 rows x 5 columns]

In [51]:
top_words_df

topic 1 topic 2 topic 3 topic 4 topic 5 topic 6 topic 7 topic 8 topic 9  \
0      物资    健健康康      快递      真的      辞职     第一天      希望      天气      疫情   
1      办公      妈妈      疫情      民生      居家      健康      上班      平安      结束   
2      端午      妞妞      花草      家有      水彩      押花      疫情      仓鼠      邻居   
3      疫情      民生      团长      封控      新闻      小区      细节      今晚      徐汇   
4      快乐      适合      果园      疫情      不行      周四      周一      家门      美景   

  topic 10 topic 11 topic 12 topic 13 topic 14 topic 15 topic 16 topic 17  \
0       东西      不要脸       这是       晃晃     胡说八道       出门       开箱       京东   
1       老板       黑市       跑腿      星期天       孩子       肉眼       一圈       物资   
2       返乡       隔离       脑残       真特       物价      连花清       离谱       一位   
3      锅包肉      人世间     载入史册      真他妈       垃圾      烦透了       网友       成本   
4       无语      守护星       天使       措施       隔离       核酸       一顿       三顿   

  topic 18 topic 19 topic 20  
0       日上      张老师      决赛圈  
1       代购      明康汇       开心  
2       感谢       喜提       礼物  
3       演员       过分       去年  
4       居家       防疫       善良

In [52]:
import pyLDAvis.sklearn
import pyLDAvis
import pyLDAvis.gensim

In [53]:
# 可视化 html 文件路径
html_path = 'document-lda-visualization.html'
# 使用 pyLDAvis 进行可视化
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
pyLDAvis.save_html(data, html_path)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path}')

#print('本次生成了文件：',
#      top_words_csv_path,
#      predict_topic_csv_path,
#      html_path)

D:\Program_Files\anaconda\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


0

In [54]:
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
#让可视化可以在notebook内显示
pyLDAvis.display(data)

In [ ]:
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51623)
Traceback (most recent call last):
  File "D:\Program_Files\anaconda\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "D:\Program_Files\anaconda\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "D:\Program_Files\anaconda\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "D:\Program_Files\anaconda\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "D:\Program_Files\anaconda\lib\http\server.py", line 427, in handle
    self.handle_one_request()
  File "D:\Program_Files\anaconda\lib\http\server.py", line 395, in handle_one_request
    self.raw_requestline = self.rfile.readline(65537)
  File "D:\Program_Files\anaconda\lib\socket.py", line 704, in readinto
    return self

## 生活满意度预测

In [ ]:
tt.head()

In [ ]:
a= tt['user_id']
b= list(set(a))
b

In [ ]:
# 写循环，把id 一致的行写入一个txt中
#模型需要输入的特征是词频比例，需要每个人的所有帖子合在一起
def pick_id(dataframe,id):
    pick = []
    all = []
    id_all = pd.DataFrame(columns=(['id','content']))
    length = len(id)
    for i in range(length):
        u_id = id[i]
        for idx in range(len(dataframe['user_id'])):
            num = dataframe['user_id'][idx]
            if num == u_id:
                pick.append(dataframe['del'][idx])
                
        sum = ""
        for aa in pick:
            sum += (aa + " ")
        all.append(sum)
        pick = []
        i += 1
    id_all['content'] = pd.DataFrame(all)
    id_all['id'] = pd.DataFrame(id)
    return id_all

In [ ]:
pi = pick_id(tt,b)
pi

In [ ]:
#特征提取
def feature_extraction_new(words):
    item = []
    #切词后词的总数
    cnt_tags = len(words)
    #print(cnt_tags)
     
    #提取每个情感词类的词频比率
    idx = 0
    for g_col in affect_col_list:
        
        #切词后，其中包含情感词的总数
        affect_cnt = 0
        
        #统计每个词类下关键词出现的总频次affect_cnt
        for i in range(cnt_tags):
            s = words[i]
            if(s in affect_dict[idx]):
                affect_cnt += 1
        
        #计算比率
        r_affect = 0.0
        if (cnt_tags > 0):
            r_affect = affect_cnt/cnt_tags
         
        item.append(r_affect)
        idx += 1
    
    return item

In [ ]:
f =  pi['content'][1]
len(f)

In [ ]:
mod_file = 'D:/ww/BDP-Resource/data/swls.mod'
clf = joblib.load(mod_file)
score = []
#word = []
for i in range(len(pi['content'])):
    uu = pi['content'][i]
    word = uu.split(' ')
    item_feature = feature_extraction_new(word)
    reshape = np.array(item_feature).reshape(1,-1)
    result = clf.predict(reshape)
    score.append(result)
    
pi['score'] = pd.DataFrame(score)

In [ ]:
pi